### This notebook is for getting popular-place-times-for-a multiple places

In [3]:
import pandas as pd
import time
import json
import datetime
import pprint
from selenium import webdriver
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

# global str
# string= "Currently 27% busy, usually 33% busy."
#https://h6f1x.sse.codesandbox.io/
import re
getPlaceIDURL='https://h6f1x.sse.codesandbox.io/' #this url is unstalbe and sometimes get changed
###############################################
#loading URL_list
df = pd.read_csv ('TimhortonsURL10.csv')
# print (df)
print(df.info())
df=df.drop_duplicates(subset = ["URL"])
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URL     5 non-null      object
dtypes: object(1)
memory usage: 168.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URL     5 non-null      object
dtypes: object(1)
memory usage: 80.0+ bytes
None


In [4]:
url_list= df["URL"].values
# print(url_list)
#####################################################################
#opernWebdriver
browser = webdriver.Firefox()
######################################################################
weekDay=["Sunday","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday"]
# a Python object (dict):

########################################### prepare document to collect the data

#
for url in url_list:
    try:
        #################################
        document = {}
        document["GoogleID"]="213424325432sdfdg"
        document["currently-occupied-by"]= 112
        document["name"]="Tim Hortons"
        document["address"]="195 Westney Rd S, Ajax, ON L1S 2C9"

        for i in weekDay:
            document[i]={}
        #################################################
        browser.get(url)
        browser.execute_script("window.scrollTo(0, 20)") 
        #############################################################
        #get the addresss
        time.sleep(4)
        data_item_id=browser.find_elements(By.XPATH,"//button[@data-item-id='address']")

        for div in  data_item_id:
            if div.get_attribute('aria-label'):
        #             print(div.get_attribute('aria-label'))
                    string=div.get_attribute('aria-label')
        #             print(string[string.find(": ")+2:])
                    string= string[string.find(": ")+2:-1]+', Canada'  #to fix the extera space at the end
                    document["address"]=string
                    print(string)
        #############################################################
        div_tag =browser.find_elements(By.TAG_NAME , "div")

        weekIndex=-1
        lastTime="p.m"
        for div in  div_tag:
            if div.get_attribute('aria-label'):
                divString=div.get_attribute('aria-label')
                if divString:
                    ####################################
                    if "a.m" in divString and "p.m" in lastTime:
                        weekIndex=weekIndex+1 ##a new day    
                    #################################### case of currently time         
                    if "Currently" in divString:
                        busy= re.findall(r'\d+',divString)[1]
                        now = datetime.now()
                        AMorPM=" a.m"
                        current_time = now.strftime("%H")
                        if int(current_time)>12:
                            current_time=int(current_time)%12
                            current_time=str(current_time)
                            AMorPM=" p.m"
                        theTime=current_time+AMorPM
                        document[weekDay[weekIndex]][theTime]=busy
                        lastTime=theTime
                    if "a.m" in divString or "p.m" in divString:
    #                     busy=int(divString[0:divString.find("%")])
                        busy=int(re.findall(r'\d+',divString)[0])
                        if "a.m" in divString:
                            AMorPM = " a.m"
                        else:
                            AMorPM = " p.m"
                        timepm= re.findall(r'\d+',divString)[1]
                        theTime= timepm+AMorPM
                        document[weekDay[weekIndex]][theTime]=busy
                        lastTime=theTime
        pprint.pprint(document)

        #############################################
        #get PlaceID using the saved Address
        browser.get(getPlaceIDURL)
        input_text= browser.find_element(By.XPATH,"//input[@id='pac-input']")
        time.sleep(15)
        input_text.click()
        input_text.send_keys('Tim Hortons, '+document["address"])
        time.sleep(10)
        input_text.send_keys(Keys.ARROW_DOWN)
        input_text.send_keys(Keys.ENTER)
        time.sleep(10)

        document["GoogleID"]=browser.find_element(By.XPATH, "//span[@id='place-id']").text
        print(document["GoogleID"])
        #############################################
        df_data =pd.DataFrame([document])
        df_data.to_csv('Data0.csv', mode='a', index=False, header=True)
    except:
        print("An exception occurred in cycle")
        print("in location address: ", document["address"]) 

745 College St, Toronto, ON M6G 1C5, Canada
{'Friday': {'1 p.m': 53,
            '10 a.m': 64,
            '10 p.m': 46,
            '11 a.m': 54,
            '12 p.m': 51,
            '2 p.m': 56,
            '3 p.m': 54,
            '4 a.m': 0,
            '4 p.m': 49,
            '5 a.m': 21,
            '5 p.m': 46,
            '6 a.m': 43,
            '6 p.m': 50,
            '7 a.m': 65,
            '7 p.m': 60,
            '8 a.m': 77,
            '8 p.m': 67,
            '9 a.m': 76,
            '9 p.m': 63},
 'GoogleID': '213424325432sdfdg',
 'Monday': {'1 p.m': 70,
            '10 a.m': 79,
            '10 p.m': 17,
            '11 a.m': 74,
            '12 p.m': 71,
            '2 p.m': 66,
            '3 p.m': 58,
            '4 a.m': 0,
            '4 p.m': 47,
            '5 a.m': 3,
            '5 p.m': 39,
            '6 a.m': 18,
            '6 p.m': 36,
            '7 a.m': 41,
            '7 p.m': 36,
            '8 a.m': 65,
            '8 p.m': 34,
            '9 a

An exception occurred in cycle
in location address:  1094 Bloor St W, Toronto, ON M6H 1M5, Canada
900 Dufferin St Unit 0565, Toronto, ON M6H 4B1, Canada
{'Friday': {'1 p.m': 33,
            '10 a.m': 36,
            '10 p.m': 0,
            '11 a.m': 28,
            '11 p.m': 0,
            '12 p.m': 29,
            '2 p.m': 34,
            '3 p.m': 31,
            '4 p.m': 25,
            '5 p.m': 18,
            '6 a.m': 0,
            '6 p.m': 11,
            '7 a.m': 0,
            '7 p.m': 6,
            '8 a.m': 31,
            '8 p.m': 0,
            '9 a.m': 42,
            '9 p.m': 0},
 'GoogleID': '213424325432sdfdg',
 'Monday': {'1 a.m': 0,
            '1 p.m': 65,
            '10 a.m': 45,
            '10 p.m': 0,
            '11 a.m': 47,
            '11 p.m': 0,
            '12 a.m': 0,
            '12 p.m': 62,
            '2 a.m': 0,
            '2 p.m': 47,
            '3 a.m': 0,
            '3 p.m': 28,
            '4 a.m': 0,
            '4 p.m': 35,
            '5 